In [1]:
from langchain_community.llms import Ollama
from langchain import PromptTemplate

model = Ollama(model='gemma2:2b', stop=["<|eot_id|>"]) # Added the stop token

/tmp/ipykernel_17086/1956216430.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model='gemma2:2b', stop=["<|eot_id|>"]) # Added the stop token


In [2]:
text = input()
user_prompt = f"{text}"

system_prompt = """
Think and write your step-by-step reasoning before responding.  
Please write only the fully spelled-out form of the acronym in English that corresponds to the following user's question, without abbreviations or additional text.  

For example, if the user asks 'RAG,' output 'Retrieval-Augmented Generation,' or for 'CNN,' output 'Convolutional Neural Network.'

If the user asks about 'MoE,' the output should be 'Mixture of Experts' and nothing else.
"""

In [3]:
template = """
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    {system_prompt}
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    {user_prompt}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=['system_prompt', 'user_prompt'],
    template=template
)

In [4]:
key_word = model(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))

/tmp/ipykernel_17086/4220479965.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  key_word = model(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))


In [5]:
key_word.strip()

'Question'

In [6]:
f"https://en.wikipedia.org/wiki/{key_word.strip()}"

'https://en.wikipedia.org/wiki/Question'

In [7]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader(f"https://en.wikipedia.org/wiki/{key_word.strip()}")
data = loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0) 
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)
# docs = vectorstore.similarity_search("")

/tmp/ipykernel_17086/1634823700.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/kar/Projects/Telegram_Chatbot/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
user_prompt = f"{user_prompt} on detail."

system_prompt = """
Please write all conversations in Korean(한국어).
Think and write your step-by-step reasoning before responding.
Write the article title using # in Markdown syntax.
Please be as specific as possible in your answer.
"""

In [10]:
from langchain.chains import RetrievalQA

qachain = RetrievalQA.from_chain_type(model, retriever=vectorstore.as_retriever())

response = qachain(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))

/tmp/ipykernel_16595/2827055214.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qachain(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


In [11]:
response

{'query': '\n    <|begin_of_text|>\n    <|start_header_id|>system<|end_header_id|>\n    \nPlease write all conversations in Korean(한국어).\nThink and write your step-by-step reasoning before responding.\nWrite the article title using # in Markdown syntax.\n\n    <|eot_id|>\n    <|start_header_id|>user<|end_header_id|>\n     MoE에 대해서 파이썬 예시 코드를 첨부해서 설명해줘 on detail.\n    <|eot_id|>\n    <|start_header_id|>assistant<|end_header_id|>\n',
 'result': '# Bad title \n\n\n\n이 질문에 대한 답변은  다음과 같습니다. \n\n1. **Wikipedia에서 특정 페이지를 검색하려면, 페이지의 주소(URL)를 입력합니다.**  예를 들어,  "https://en.wikipedia.org/wiki/Special:Badtitle" 를 사용하면 Wikipedia에 "Bad Title" 페이지로 이동할 수 있습니다. \n2. **페이지 URL에서 특정 제목을 찾고 싶다면, \n\n    * 웹사이트의 주소가 필요합니다. \n    * 해당 웹사이트에서 "Bad title" 을 검색하세요. \n\n\n\n**이 글은 사용자가 도움말을 요청하는 것을 위한 예시입니다.** \n\n'}

In [12]:
print(response['result'])

# Bad title 



이 질문에 대한 답변은  다음과 같습니다. 

1. **Wikipedia에서 특정 페이지를 검색하려면, 페이지의 주소(URL)를 입력합니다.**  예를 들어,  "https://en.wikipedia.org/wiki/Special:Badtitle" 를 사용하면 Wikipedia에 "Bad Title" 페이지로 이동할 수 있습니다. 
2. **페이지 URL에서 특정 제목을 찾고 싶다면, 

    * 웹사이트의 주소가 필요합니다. 
    * 해당 웹사이트에서 "Bad title" 을 검색하세요. 



**이 글은 사용자가 도움말을 요청하는 것을 위한 예시입니다.** 


